In [1]:
import os
os.chdir('../')
os.getcwd()
from old_type.TwoStageTrAdaBoostR2 import *
from old_type.TwoStageTrAdaBoostR2_new import *
from old_type.TransferStacking import TransferStacking

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import arff

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso

In [3]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter('ignore', ConvergenceWarning)

In [4]:
def make_dataset_concrete():
    data = arff.loadarff('./dataset/transfer data/new-concrete1.arff')
    df1 = pd.DataFrame(data[0])
    df1["label"] = 0
    data = arff.loadarff('./dataset/transfer data/new-concrete2.arff')
    df2 = pd.DataFrame(data[0])
    df2["label"] = 1
    data = arff.loadarff('./dataset/transfer data/new-concrete3.arff')
    df3 = pd.DataFrame(data[0])
    df3["label"] = 2
    df = pd.concat([df1, df2, df3], sort=False).reset_index(drop=True)
    return df

In [5]:
df = make_dataset_concrete()
df

,b,c,d,e,f,g,h,i,label
0,153.0,0.0,192.0,0.0,887.0,942.0,3.0,4.57,0
1,153.0,0.0,192.0,0.0,887.0,942.0,90.0,25.46,0
2,153.0,0.0,192.0,0.0,887.0,942.0,7.0,7.68,0
3,153.0,0.0,192.0,0.0,887.0,942.0,28.0,17.28,0
4,162.4,0.0,203.5,0.0,938.2,849.0,28.0,20.59,0
...,...,...,...,...,...,...,...,...,...
1025,0.0,0.0,173.0,0.0,1125.0,613.0,28.0,67.31,2
1026,0.0,0.0,173.0,0.0,1125.0,613.0,90.0,69.66,2
1027,0.0,0.0,173.0,0.0,1125.0,613.0,180.0,71.62,2
1028,0.0,0.0,173.0,0.0,1125.0,613.0,270.0,74.17,2


In [6]:
n_target=25
y_feature="i"

In [7]:
max_depth = 3
n_estimators = 30
steps =30
fold = 10
random_state = np.random.RandomState(1)
base_estimator = DecisionTreeRegressor(max_depth=max_depth)
meta_model = Lasso(alpha=0.1)

In [10]:
target_domain = 0

target_train = df.query('label==@target_domain').sample(n_target)
target_test = df.query('label==@target_domain').drop(index=target_train.index)
target_train_X = target_train.drop(columns=[y_feature, 'label'])
target_train_y = target_train[y_feature]
target_test_X = target_test.drop(columns=[y_feature, 'label']).values
target_test_y = target_test[y_feature].values
source_X = df.query('label!=@target_domain').drop(columns=[y_feature, 'label'])
source_y = df.query('label!=@target_domain')[y_feature]

target_train_label = target_train['label']
source_label =df.query('label!=@target_domain')['label']
domain_col = pd.concat([target_train_label, source_label]).values

X_train = pd.concat([source_X, target_train_X]).values
y_train = pd.concat([source_y, target_train_y]).values
sample_size = [source_X.shape[0], target_train_X.shape[0]]


# two-stage
regr_tr = TwoStageTrAdaBoostR2(base_estimator,
                            sample_size=sample_size, 
                            n_estimators = n_estimators,
                            steps = steps, fold = fold,
                            random_state = random_state)
regr_tr.fit(X_train, y_train)
y_pred_tr = regr_tr.predict(target_test_X)
rmse_tr = mean_squared_error(target_test_y, y_pred_tr)**0.5

#  new two-stage
regr_tr_new = NewTwoStageTrAdaBoostR2(base_estimator,
                            n_estimators = n_estimators,
                            steps = steps, fold = fold,
                            random_state = random_state)
regr_tr_new.fit(X_train, y_train, domain_col, target_domain)
y_pred_tr_new = regr_tr_new.predict(target_test_X)
rmse_tr_new = mean_squared_error(target_test_y, y_pred_tr_new)**0.5

print('旧 : {:.4}'.format(rmse_tr))
print('新 : {:.4}'.format(rmse_tr_new))

旧 : 10.79
新 : 10.38


In [25]:
def experimence(df, target_domain, base_estimator):
    results = []
    for count in range(30):
        print(str(count).center(100,"-"))

        target_train = df.query('label==@target_domain').sample(n_target)
        target_test = df.query('label==@target_domain').drop(index=target_train.index)
        target_train_X = target_train.drop(columns=[y_feature, 'label'])
        target_train_y = target_train[y_feature]
        target_test_X = target_test.drop(columns=[y_feature, 'label']).values
        target_test_y = target_test[y_feature].values
        source_X = df.query('label!=@target_domain').drop(columns=[y_feature, 'label'])
        source_y = df.query('label!=@target_domain')[y_feature]

        target_train_label = target_train['label']
        source_label =df.query('label!=@target_domain')['label']
        domain_col = pd.concat([source_label, target_train_label])

        X_train = pd.concat([source_X, target_train_X])
        y_train = pd.concat([source_y, target_train_y])
        sample_size = [source_X.shape[0], target_train_X.shape[0]]


        # two-stage
        regr_tr = TwoStageTrAdaBoostR2(base_estimator,
                                    sample_size=sample_size, 
                                    n_estimators = n_estimators,
                                    steps = steps, fold = fold,
                                    random_state = random_state)
        regr_tr.fit(X_train.values, y_train.values)
        y_pred_tr = regr_tr.predict(target_test_X)
        rmse_tr = mean_squared_error(target_test_y, y_pred_tr)**0.5

        #  new two-stage
        regr_tr_new = NewTwoStageTrAdaBoostR2(base_estimator,
                                    n_estimators = n_estimators,
                                    steps = steps, fold = fold,
                                    random_state = random_state)
        regr_tr_new.fit(X_train, y_train, domain_col, target_domain)
        y_pred_tr_new = regr_tr_new.predict(target_test_X)
        rmse_tr_new = mean_squared_error(target_test_y, y_pred_tr_new)**0.5
        
        #  new two-stage
        regr_tr_shuffle = NewTwoStageTrAdaBoostR2(base_estimator,
                                    n_estimators = n_estimators,
                                    steps = steps, fold = fold,
                                    random_state = random_state)
        n = len(y_train)
        regr_tr_shuffle.fit(X_train.sample(n=n, random_state=0), 
                            y_train.sample(n=n, random_state=0), 
                            domain_col.sample(n=n, random_state=0), target_domain)
        y_pred_tr_shuffle = regr_tr_shuffle.predict(target_test_X)
        rmse_tr_shuffle = mean_squared_error(target_test_y, y_pred_tr_shuffle)**0.5

        print('旧 : {:.4}'.format(rmse_tr))
        print('新 : {:.4}'.format(rmse_tr_new))
        print('新(シャッフル) : {:.4}'.format(rmse_tr_new))

        results.append([rmse_tr, rmse_tr_new, rmse_tr_shuffle])
    return pd.DataFrame(results, columns=["旧", "新", "新(シャッフル)"])
#         results.append([rmse_targetada, rmse_ada, rmse_stacked])
#     return pd.DataFrame(results, columns=["target", "all", "stacking"])

In [26]:
max_depth = 3

df_results_low = experimence(df, 0, base_estimator)
df_results_medium = experimence(df, 1, base_estimator)
df_results_high = experimence(df, 2, base_estimator)

results_dt = pd.concat([df_results_low.mean(), df_results_medium.mean(), df_results_high.mean()], axis=1).T
results_dt

-------------------------------------------------0--------------------------------------------------
旧 : 13.41
新 : 14.29
新(シャッフル) : 14.29
-------------------------------------------------1--------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 11.53
新 : 10.05
新(シャッフル) : 10.05
-------------------------------------------------2--------------------------------------------------
旧 : 12.52
新 : 14.91
新(シャッフル) : 14.91
-------------------------------------------------3--------------------------------------------------
旧 : 12.44
新 : 13.21
新(シャッフル) : 13.21
-------------------------------------------------4--------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is s

At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 9.358
新 : 8.175
新(シャッフル) : 8.175
-------------------------------------------------18-------------------------------------------------
At step: 28
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 9.786
新 : 9.468
新(シャッフル) : 9.468
-------------------------------------------------19--------------------------------------------

旧 : 12.23
新 : 10.94
新(シャッフル) : 10.94
-------------------------------------------------4--------------------------------------------------
At step: 28
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 12.96
新 : 12.72
新(シャッフル) : 12.72
-------------------------------------------------5--------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 

At step: 28
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 8.956
新 : 9.269
新(シャッフル) : 9.269
-------------------------------------------------26-------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 9.828
新 : 9.956
新(シャッフル) : 9.956
-------------------------------------------------27--------------------------------------------

At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 15.36
新 : 15.37
新(シャッフル) : 15.37
-------------------------------------------------14-------------------------------------------------
At step: 29
Binary search's goal not meeted! Value is set to be the available best!
Try reducing the search interval. Current stp interval: 1e-30
旧 : 14.24
新 : 14.41
新(シャッフル) : 14.41
-------------------------------------------------15-------------------------------------------------
旧 : 13.87
新 : 14.01
新(シャッフル) : 14.01
-------------------------------------------------16-------------------------------------------------
旧 : 12.8
新 : 12.28
新(シャッフル) : 12.28
-------------------------------------------------17-------------------------------------------------
旧 : 13.88
新 : 14.38
新(シャッフル) : 14.38
-------------------------------------------------18-------------------------------------------------
At step: 29
Binary 

,旧,新,新(シャッフル)
0,10.427452,10.395889,10.327851
1,10.039553,9.952537,10.235857
2,13.879754,13.757478,13.523707
